In [1]:
# 2) Imports
import os
import re
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [2]:
# 3) Load your data
csv_file = "variant_sample_25k.csv"
assert os.path.exists(csv_file), f"CSV not found: {csv_file}"

df = pd.read_csv(csv_file)
df.head()


,ReferenceAllele,AlternateAllele,Chromosome,Start,Stop,GeneSymbol,ClinicalSignificance,PhenotypeList,Type,Assembly
0,na,na,7,4820844,4820847,AP5Z1,Pathogenic/Likely pathogenic,Hereditary spastic paraplegia 48|Macular dystr...,Indel,GRCh37
1,na,na,7,4781213,4781216,AP5Z1,Pathogenic/Likely pathogenic,Hereditary spastic paraplegia 48|Macular dystr...,Indel,GRCh38
2,na,na,7,4827361,4827374,AP5Z1,Pathogenic,Hereditary spastic paraplegia 48,Deletion,GRCh37
3,na,na,7,4787730,4787743,AP5Z1,Pathogenic,Hereditary spastic paraplegia 48,Deletion,GRCh38
4,na,na,15,85342440,85342440,ZNF592,Uncertain significance,Galloway-Mowat syndrome 1,single nucleotide variant,GRCh37


In [3]:
# 4) Drop duplicates and obvious bad rows (defensive)
df = df.drop_duplicates()
df = df.dropna(subset=["ReferenceAllele","AlternateAllele","Chromosome","Start","GeneSymbol","ClinicalSignificance"])
df = df.reset_index(drop=True)

# Normalize whitespace
for col in ["ReferenceAllele","AlternateAllele","GeneSymbol","ClinicalSignificance","PhenotypeList","Type","Assembly"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()


In [4]:
# 5) Map ClinicalSignificance to a canonical single class
# You can adjust this mapping to fit your modeling goal.
canonical_map = {
    "Pathogenic": "Pathogenic",
    "Likely pathogenic": "Likely pathogenic",
    "Benign": "Benign",
    "Likely benign": "Likely benign",
    "Uncertain significance": "VUS",  # VUS = Variant of Uncertain Significance
}

def normalize_clinsig(x: str) -> str:
    s = x.lower()
    # Prioritize specific terms; order matters
    if "pathogenic" in s and "likely" not in s and "conflict" not in s:
        return "Pathogenic"
    if "likely pathogenic" in s:
        return "Likely pathogenic"
    if "benign" in s and "likely" not in s and "conflict" not in s:
        return "Benign"
    if "likely benign" in s:
        return "Likely benign"
    if "uncertain" in s:
        return "VUS"
    # collapse everything else into "Other" to avoid label explosion
    return "Other"

df["ClinSigClass"] = df["ClinicalSignificance"].apply(normalize_clinsig)

# Option: drop "Other" to simplify; or keep it as a class
keep_classes = ["Pathogenic","Likely pathogenic","Benign","Likely benign","VUS"]
df = df[df["ClinSigClass"].isin(keep_classes)].reset_index(drop=True)

df["ClinSigClass"].value_counts()


ClinSigClass
Pathogenic           15792
Likely pathogenic     4656
VUS                   1345
Benign                 511
Likely benign          378
Name: count, dtype: int64

In [5]:
# 6) Simple featurization of alleles
def allele_features(series):
    # length and nucleotide composition
    out = pd.DataFrame({
        "len": series.str.len().fillna(0).astype(int),
        "count_A": series.str.upper().str.count("A").fillna(0).astype(int),
        "count_C": series.str.upper().str.count("C").fillna(0).astype(int),
        "count_G": series.str.upper().str.count("G").fillna(0).astype(int),
        "count_T": series.str.upper().str.count("T").fillna(0).astype(int),
    })
    return out

ref_feats = allele_features(df["ReferenceAllele"]).add_prefix("ref_")
alt_feats = allele_features(df["AlternateAllele"]).add_prefix("alt_")

# Make Chromosome numeric-like: X,Y,MT -> 23,24,25
def chr_to_int(x):
    s = str(x).strip()
    if s in ["X","x"]: return 23
    if s in ["Y","y"]: return 24
    if s.upper() in ["MT","M","MITO","CHRMT"]: return 25
    try:
        v = int(s)
        return v if 1 <= v <= 22 else 0
    except:
        return 0

df["ChromInt"] = df["Chromosome"].apply(chr_to_int).astype(int)

# Start/Stop numeric
df["Start"] = pd.to_numeric(df["Start"], errors="coerce")
if "Stop" in df.columns:
    df["Stop"] = pd.to_numeric(df["Stop"], errors="coerce")
else:
    df["Stop"] = df["Start"]  # fallback if Stop absent in some rows

# GeneSymbol: cap cardinality
top_k = 3000  # adjust based on memory/training time
top_genes = df["GeneSymbol"].value_counts().nlargest(top_k).index
df["GeneSymbol_Limited"] = np.where(df["GeneSymbol"].isin(top_genes), df["GeneSymbol"], "OTHER")

# Select raw columns for preprocessing
num_cols = [
    "ChromInt","Start","Stop",
    "ref_len","ref_count_A","ref_count_C","ref_count_G","ref_count_T",
    "alt_len","alt_count_A","alt_count_C","alt_count_G","alt_count_T"
]
cat_cols = []
if "Type" in df.columns: cat_cols.append("Type")
if "Assembly" in df.columns: cat_cols.append("Assembly")
cat_cols.append("GeneSymbol_Limited")

# Assemble feature frame
X_num = pd.concat([ref_feats, alt_feats], axis=1)
X_base = pd.concat([df[["ChromInt","Start","Stop"]], X_num], axis=1)
X_cat = df[cat_cols].copy()

X = pd.concat([X_base, X_cat], axis=1)
y = df["ClinSigClass"].copy()

X.head(), y.head()


(   ChromInt     Start      Stop  ref_len  ref_count_A  ref_count_C  \
 0         7   4820844   4820847        2            1            0   
 1         7   4781213   4781216        2            1            0   
 2         7   4827361   4827374        2            1            0   
 3         7   4787730   4787743        2            1            0   
 4        15  85342440  85342440        2            1            0   
 
    ref_count_G  ref_count_T  alt_len  alt_count_A  alt_count_C  alt_count_G  \
 0            0            0        2            1            0            0   
 1            0            0        2            1            0            0   
 2            0            0        2            1            0            0   
 3            0            0        2            1            0            0   
 4            0            0        2            1            0            0   
 
    alt_count_T                       Type Assembly GeneSymbol_Limited  
 0            0  

In [6]:
# 7) Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)


(ClinSigClass
 Pathogenic           0.696225
 Likely pathogenic    0.205291
 VUS                  0.059300
 Benign               0.022541
 Likely benign        0.016644
 Name: proportion, dtype: float64,
 ClinSigClass
 Pathogenic           0.696275
 Likely pathogenic    0.205202
 VUS                  0.059290
 Benign               0.022482
 Likely benign        0.016751
 Name: proportion, dtype: float64)

In [8]:
# 8) Preprocessing with ColumnTransformer
numeric_features = num_cols
categorical_features = cat_cols

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler(with_mean=True, with_std=True))
])

categorical_transformer = OneHotEncoder(handle_unknown="ignore", dtype=np.float32)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Fit on train, transform train/val
preprocessor.fit(X_train)

X_train_proc = preprocessor.transform(X_train)
X_val_proc = preprocessor.transform(X_val)

X_train_proc.shape, X_val_proc.shape


((18145, 1859), (4537, 1859))

In [9]:
import joblib
joblib.dump(preprocessor, "preprocessor_clinsig.joblib")


['preprocessor_clinsig.joblib']

In [10]:
# 9) Label encoding
label_encoder = OrdinalEncoder(dtype=np.int32)
y_train_enc = label_encoder.fit_transform(y_train.to_frame()).astype(np.int32).ravel()
y_val_enc = label_encoder.transform(y_val.to_frame()).astype(np.int32).ravel()

classes_ = label_encoder.categories_[0].tolist()
num_classes = len(classes_)
classes_


['Benign', 'Likely benign', 'Likely pathogenic', 'Pathogenic', 'VUS']

In [11]:
# 10) Class weights to mitigate imbalance
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(num_classes),
    y=y_train_enc
)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}
class_weight_dict


{0: 8.87286063569682,
 1: 12.016556291390728,
 2: 0.9742281879194631,
 3: 0.2872635161877622,
 4: 3.3726765799256504}

In [12]:
# 11) Build model
tf.keras.utils.set_random_seed(42)
input_dim = X_train_proc.shape[1]

def build_mlp(input_dim, num_classes, dropout=0.2):
    inputs = keras.Input(shape=(input_dim,))
    x = layers.Dense(512, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_mlp(input_dim, num_classes, dropout=0.3)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1859)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       952,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,120,773 (4.28 MB)

 Trainable params: 1,118,981 (4.27 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [13]:
# 12) Train
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=5, restore_best_weights=True
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6
)

history = model.fit(
    X_train_proc, y_train_enc,
    validation_data=(X_val_proc, y_val_enc),
    epochs=50,
    batch_size=512,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.2049 - loss: 2.2547 - val_accuracy: 0.0595 - val_loss: 1.6563 - learning_rate: 0.0010
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2755 - loss: 1.7344 - val_accuracy: 0.0719 - val_loss: 1.6746 - learning_rate: 0.0010
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3243 - loss: 1.4429 - val_accuracy: 0.0679 - val_loss: 1.6876 - learning_rate: 0.0010
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3701 - loss: 1.2634 - val_accuracy: 0.0961 - val_loss: 1.5789 - learning_rate: 0.0010
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4146 - loss: 1.0808 - val_accuracy: 0.1087 - val_loss: 1.5746 - learning_rate: 0.0010
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4605 - loss: 0.9764 - val_accuracy: 0.1913 - val_loss: 1.5058 - learning_rate: 0.0010
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4998 - loss: 0.8869 - val_acc

In [14]:
# 13) Evaluation
val_probs = model.predict(X_val_proc, batch_size=1024)
val_pred = np.argmax(val_probs, axis=1)

print("Accuracy:", (val_pred == y_val_enc).mean())

print("\nClassification report:")
print(classification_report(y_val_enc, val_pred, target_names=classes_))

print("\nConfusion matrix:")
print(confusion_matrix(y_val_enc, val_pred))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Accuracy: 0.5080449636323562

Classification report:
                   precision    recall  f1-score   support

           Benign       0.33      0.51      0.40       102
    Likely benign       0.09      0.37      0.14        76
Likely pathogenic       0.36      0.61      0.45       931
       Pathogenic       0.88      0.48      0.62      3159
              VUS       0.18      0.51      0.26       269

         accuracy                           0.51      4537
        macro avg       0.37      0.50      0.38      4537
     weighted avg       0.71      0.51      0.55      4537


Confusion matrix:
[[  52    5   15    3   27]
 [  10   28   15    6   17]
 [  22   48  566  153  142]
 [  66  215  913 1522  443]
 [   9   28   59   36  137]]


In [18]:
# 15) Inference function
import joblib
def preprocess_new(df_new: pd.DataFrame):
    # reproduce feature engineering
    ref_feats = allele_features(df_new["ReferenceAllele"]).add_prefix("ref_")
    alt_feats = allele_features(df_new["AlternateAllele"]).add_prefix("alt_")
    df_new["ChromInt"] = df_new["Chromosome"].apply(chr_to_int).astype(int)
    df_new["Start"] = pd.to_numeric(df_new["Start"], errors="coerce")
    if "Stop" not in df_new.columns:
        df_new["Stop"] = df_new["Start"]
    else:
        df_new["Stop"] = pd.to_numeric(df_new["Stop"], errors="coerce")

    # gene bucketing: unseen -> OTHER
    df_new["GeneSymbol_Limited"] = np.where(df_new["GeneSymbol"].isin(top_genes), df_new["GeneSymbol"], "OTHER")

    X_num = pd.concat([ref_feats, alt_feats], axis=1)
    X_base = pd.concat([df_new[["ChromInt","Start","Stop"]], X_num], axis=1)
    X_cat = df_new[cat_cols].copy()

    X_new = pd.concat([X_base, X_cat], axis=1)
    pre = joblib.load("preprocessor_clinsig.joblib")
    return pre.transform(X_new)

loaded_model = keras.models.load_model("clinsig_mlp")
with open("label_encoder_classes.json") as f:
    classes_loaded = json.load(f)

# Example (make sure columns exist)
# df_example = df.iloc[:5].copy()
# X_example = preprocess_new(df_example)
# preds = loaded_model.predict(X_example)
# pred_labels = np.array(classes_loaded)[np.argmax(preds, axis=1)]
# pred_labels


ValueError: File format not supported: filepath=clinsig_mlp. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(clinsig_mlp, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).